# Introduction à la fouille de texte en Python: la densité lexicale

## Introduction

Maintenant qu'on a vu les bases de Python, on va avancer un peu vers de l'analyse computationnelle plus poussée.

Il s'agira de comparer trois romans de Virginia Woolf: *Mrs. Dalloway* (1925), *To the lighthouse* (1927), *The Waves* (1931). On va voir si le style de Woolf évolue, et si oui, comment. On analyse les textes par "lecture distante" (par opposition à une lecture "proche", humaine): on ne *lit pas* les romans, mais on les prend comme corpus de données qui peuvent être approchées par des méthodes computationnelles. On peut donc travailler sur de grands corpus, ici de trois romans entiers:

- **Mrs. Dalloway** décrit la journée de Clarissa Dalloway, une femme britannique de classe sociale supérieure, avant la 1e Guerre Mondiale. Le roman suit le flux de conscience de Clarissa, et il est souvent considéré comme une réponse à *Ulysses* de Joyce, qui suit lui aussi un personnage pendant une journée (livre qu'elle critique abondamment par ailleurs).
- **To the lighthouse** décrit quelques journées de la famille Ramsay et de leurs invité.e.s dans leur maison de villégiature, sur l'île de Syke en Écosse. Le roman s'étale sur une période de 10 ans, avant et après la Première Guerre Mondiale. Il entrecroise différentes histoires familiales, contient peu de dialogues directs et se concentre sur les pensées des personnages.
- **The Waves** est composé de solliloques et dialogues de six narrateur.ice.s qui s'étalent sur plusieurs années, de l'enfance à l'âge adulte. C'est généralement considéré comme son roman le plus expérimental, et le plus difficile à lire.

Notre question de recherche:

> Le style de Woolf évolue-t-il d'un roman à l'autre?
> Est-ce que le caractère "expérimental" et "difficile" de *The Waves* ressort d'une analyse statistique ?

Le choix du corpus, et une partie de la méthodologie, sont inspirés de:

> Hussein, K. & Kadhim, R. (2020). A Corpus-Based Stylistic Identification of Lexical Density Profile of Three Novels by Virginia Woolf: The Waves, Mrs. Dalloway and To the Lighthouse. *International Journal of Psychosocial Rehabilitation*. 24. pp. 6688-9702. En accès libre à [cette addresse](https://www.researchgate.net/publication/343797320_A_Corpus-Based_Stylistic_Identification_of_Lexical_Density_Profile_of_Three_Novels_by_Virginia_Woolf_The_Waves_Mrs_Dalloway_and_To_the_Lighthouse)

---
---

## Le programme des festivités

### Pipeline

Ces trois romans ont été téléchargés depuis Archive.org en texte brut (c'est-à-dire, sans éléments de mise en page) et légèrement nettoyés (les en-têtes, fin de page, la pagination et la séparation de chapitres sont supprimés). La chaîne de traitement est la suivante:

- **on ouvre les fichiers** et on lit leur contenu
- **on étudie le vocabulaire**: d'abord par un POS-tagging, ensuite un calcul de sa *densité lexicale*

### Compétences

Notre analyse permet d'introduire au TAL et aux bases de l'utilisation de Python pour la fouille de texte:

- **installation et utilisation de librairies**
- **traitement automatisé du langage, avec `nltk`** (enfin, une très très très rapide introduction à quelques concepts de TAL): *part-of-speech tagging*, calcul de densité lexicale.

---
---

## Les librairies

On commence par installer les **librairies** nécessaires. 

> En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des libraires.
> - Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul...
> - Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée "module" ou "package".

Les fichiers sont installés avec la commande `pip install <package name>` dans un terminal, ou `!pip install` dans un notebook.

In [ ]:
!pip install nltk==3.9.1

Ensuite, on importe les libraires nécessaires pour les utiliser dans notre chaîne de traitement.

In [ ]:
import nltk

Pour importer une librairie, on utilise la commande `import`:

```python
import nom_du_paquet
```

Les librairies sont souvent organisées en *sous-modules* (comme les chapitres d'un livre). Il est donc aussi possible d'importer seulement un *module*, ou une seule fonction:

```python
from nom_du_paquet import nom_du_module
from nom_du_paquet.nom_du_module import nom_du_module_ou_fonction
```

Ensuite, `nltk` (la librairie faite pour le traitement du langage naturel) demande de faire quelques téléchargements:



In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger_eng')

---

# Lire les fichiers

(Attention, il faut que les [trois fichiers](https://github.com/paulhectork/cours_ens2024_fouille_de_texte/tree/main/in) aient été téléchargés au format `.txt` dans la partie `Fichiers` de Google Collab, sans les renommer !)

On commence donc par lire le contenu de nos textes. Comme on a 3 textes sur lesquels on va vouloir mener les mêmes opérations, on définit des fonctions.

In [ ]:
def lire(nom_de_fichier):
    with open(nom_de_fichier, mode="r" ) as fh:
        corpus = fh.read()
    return corpus

In [ ]:
# qu'est-ce qui se passe ici?
dalloway = lire("in_mrs_dalloway.txt")
lighthouse = lire("in_to_the_lighthouse.txt")
waves = lire("in_the_waves.txt")

print(waves[:1000])

---

# Étudier le vocabulaire: la *densité lexicale*

## Définitions

Jusqu'à maintenant, on est resté sur des méthodes d'analyse assez basiques. C'est vraiment moins de l'analyse que de la restructuration du texte. Ça nous a permis d'introduire quelques bases de Python, mais on est assez loin de produire quelque chose de vraiment intéressant.

À partir de maintenant, on va utiliser des méthodes de TAL, avec la librairie Python dédiée, `nltk`. Il existe tout plein de méthodes de traitement du langage (reconnaissance d'entités nommées, *topic modelling*, *sentiment analysis*...)

L'avantage d'une librairie, c'est que ça prémache complètement le travail. Le désavantage, c'est qu'il faut savoir utiliser une librairie (quelles fonctions utiliser, comment etc.). Pour savoir comment les utiliser, on se réfère à la [documentation](https://www.nltk.org/api/nltk.tokenize.word_tokenize.html). La documentation, c'est parfois obscur. Apprendre à lire de la doc c'est donc un travail à part entière quand on apprend à programmer.

Manipulons rapidement le texte avec deux fonctions de `nltk`:

In [ ]:
# tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)
tokens = nltk.word_tokenize(waves)
print(tokens)
print(type(tokens))

# tokens est une liste. comment affichier le nombre de tokens dans the waves ?

In [ ]:
# pos_tag = part-of-speech tagging. on verra se qui se passe plus en détail plus tard
print( nltk.pos_tag(tokens, tagset="universal") )

# est-ce que vous avez une idée de ce qui s'est passé au dessus ?

Vous l'aurez vu, avec le [`POS tag`](https://fr.wikipedia.org/wiki/%C3%89tiquetage_morpho-syntaxique) (part-of-speech tagging, ou étiquetage morphosyntaxique), on peut classer les mots du texte en fonction du rôle qu'ils occupent: verbe, déterminants, etc. Ça permet des analyses beaucoup plus détaillées, puisqu'on a un tagging assez fin de chaque mot. Par exemple, si on veut s'intéresser à l'usage des déterminants, on peut ne sélectionner que les *tokens* taggués avec `DET`.

L'argument `tagset` permet de préciser les types de tags à produire. *Universal* définit un jeu de tags assez générique (il en existe des plus précis, mais inutiles ici):

```
Tag  | Meaning             | English Examples
~~~~~|~~~~~~~~~~~~~~~~~~~~~|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ADJ  | adjective           | new, good, high, special, big, local
ADP  | adposition          | on, of, at, with, by, into, under
ADV  | adverb              | really, already, still, early, now
CONJ | conjunction         | and, or, but, if, while, although
DET  | determiner, article | the, a, some, most, every, no, which
NOUN | noun                | year, home, costs, time, Africa
NUM  | numeral             | twenty-four, fourth, 1991, 14:24
PRT  | particle            | at, on, out, over per, that, up, with
PRON | pronoun             | he, their, her, its, my, I, us
VERB | verb                | is, say, told, given, playing, would
.    | punctuation marks   | . , ; !
X    | other               | ersatz, esprit, dunno, gr8, univeristy
```

On peut maintenant estimer la proportion de verbes dans le texte, par exemple:

In [ ]:
tokens_tags = nltk.pos_tag(tokens, tagset="universal")

verb_count = 0
for (token, tag) in tokens_tags:
    if tag == "VERB":
        verb_count += 1
print(verb_count)
        
# et maintenant, comment est-ce qu'on calcule la proportion de verbes ? 

Ce calcul au dessus, c'est la base de la [densité lexicale](https://en.wikipedia.org/wiki/Lexical_density) ! 

La densité lexicale vise à mesurer la complexité informationnelle d'un texte, écrit ou oral. Elle s'appuie sur la classification des mots en deux catégories:
- les **mots-lexicaux** (`lexical units`) regroupent l'ensemble de mots "porteurs d'information" dans un texte: les noms, les verbes, les adverbes et les adjectifs qualitatifs. On considère que c'est grâce à eux que l'information est transmise.
- les **mots-outils ou mots-grammaticaux** (`function words`) regroupent tout le reste des mots. On considère qu'ils sont moins porteurs de sens et qu'ils servent surtout à la structure de la phrase

La classification entre ces deux groupes n'est pas clairement définie et peut varier d'une étude à l'autre.

La **densité lexicale** (notée $L_d$) mesure, sur une échelle de 0 à 100, la proportion de mots lexicaux dans un texte. 

Elle se mesure en  calculant la proportion de *mots lexicaux* par rapport au nombre total de mots. Soit $N_{lex}$ le nombre de mots lexicaux et $N$ le nombre total de mots dans un texte:

$$L_d = (\frac{N_{lex}}{N}) \times 100$$

Plus $L_d$ est proche de 100, plus il y a de mots lexicaux, plus on considère que le texte est riche en information. On considère en moyenne que que:
- pour un texte écrit, $L_d > 40$, et pour un texte oral, $L_d < 40$.
- pour un texte de fiction, $ 40 < L_d < 54$; pour un texte de non-fiction, $40 < L_d < 65$.

PS: il existe plusieurs variantes de calcul, mais on utilise celle définie par Ure J. (1971).

### Chaîne de traitement

Avec `nltk`, calculer une densité lexicale est vraiment très très simple:

- on tokenise notre texte en `tokens` (items lexicaux)
- on fait un *POS tag*
- on compte le nombre d'occurrences pour chaque fonction des tokens d'un texte
- à partir de là, on calcule notre $L_d$

---

Pour commencer, on généralise ce qui avait déjà été fait au dessus: on avait travaillé seulement sur les verbes, maintenant on travaille sur tout.

In [ ]:
# on reprend le POS tagging
tokens = nltk.word_tokenize(waves)  # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)
print(tokens[:10])

size = len(tokens)                              # la taille du corpus
pos = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging. universal définit des classes très généralistes
print(pos[:10])                                 # que voit-on ici?
print(size)

Ensuite, on regarde le nombre d'occurrences de chaque tag. On pourrait faire un dict qui associe chaque tag à son nombre d'occurrences, mais NLTK mâche le travail en faisant des distributions de fréquences à partir de listes de valeurs.

> En statistiques, une distribution de fréquences, c'est un tableau qui associe les différentes valeurs d'un échantillon à la fréquence à laquelle ces valeurs apparaissent.


In [ ]:
tags = []
for (token, tag) in pos:
    tags.append(tag)

fd = nltk.FreqDist(tags)  # `nltk.FreqDist()` associe à chaque valeur distincte d'une liste le nombre d'occurrences dans cette liste
fd.tabulate()

On est presque prêt.e.s ! Il suffit maintenant d'additionner toutes les fréquences de toutes les unités lexicales (noms, verbes...) et de comparer au reste du corpus. 

In [ ]:
# fd.get() permet de récuperer la fréquence de la valeur entre parenthèses
nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales

print("taille du corpus          :", size)
print("nombre de mots-outils     :", size - nlex)
print("nombre d'unités lexicales :", nlex)

In [ ]:
ld = (nlex/size) * 100
print(ld)

On a maintenant la densité lexicale de *The Waves* ! On écrit une fonction qui reprenne tout ça et on la lance:

In [ ]:
def densite_lexicale(text):
    """
    enfin, on étudie la densité lexicale de chaque roman

    on suit la méthode de Ure: 100 * <nb d'unités lexicales> / <nb de tokens>
    https://en.wikipedia.org/wiki/Lexical_density
    https://www.nltk.org/book/ch05.html
    """
    tokens = nltk.word_tokenize(text)    # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)

    size = len(tokens)  # on travaille sur tout le corpus
    tokens_tags = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
    tags = []
    for (token, tag) in tokens_tags:
        tags.append(tag)

    fd = nltk.FreqDist(tags)  # valeur associée aux nombre d'occurrences de celle-ci
    nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales
    ld = 100 * (nlex/size)

    return ld


ld_lighthouse = densite_lexicale(lighthouse)
ld_dalloway = densite_lexicale(dalloway)
ld_waves = densite_lexicale(waves)

print("Mrs Dalloway      (1925):", ld_dalloway)
print("To the lighthouse (1927):", ld_lighthouse)
print("The Waves         (1931):", ld_waves)


Les résultats ne sont pas *criants*, mais on voit quand même clairement que *To the lighthouse* marque un net déclin dans la densité lexicale, et que *The Waves*, censé être le plus difficile et expérimental des trois livres, a une densité lexicale supérieure. À mon humble avis, *To the lighthouse* n'est pas une lecture évidente pour autant, même si elle vaut le coup d'être faite :) 

La densité lexicale n'est qu'une mesure, et elle est elle-même très limitée. Cependant, on voit maintenant qu'en une fonction et moins de 50 lignes de code qu'on arrive à faire une étude quantitative complète sur trois livres, de façon quasi instantannée !

---

## Pour aller plus loin

J'ai quelques autres notebooks qui peuvent être utiles:

- [Cours donné en 2024 à l'ENS](https://github.com/paulhectork/cours_ens2024_fouille_de_texte) (2h). C'est le même corpus de trois romans de Virginia Woolf, mais en plus de la densité lexicale, on voit l'analyse de la structure des phrases et paragraphes, ainsi que la distribution du vocabulaire dans les trois romans pour chercher à mener une analyse statistique un peu plus poussée.
- [Atelier donné en 2023 à l'ENS](https://github.com/paulhectork/cours_ens2023_xmltei) (1 journée) sur l'analyse de texte en XML. On y voit le XML-TEI, un standard de description de documents, et on y fait du TAL beaucoup plus avancé: géocodage, résolutions d'entités nommées... Bon ce cours est peut-être un peu compliqué mais je le mets quand même:) 


Ces ressources pédagogiques sont aussi très utiles et m'ont servi à préparer ces cours:
- Laramée, F. D. (2018). Introduction à la stylométrie en Python. *Programming historian*. [En ligne](https://programminghistorian.org/fr/lecons/introduction-a-la-stylometrie-avec-python)
- Lavin, Matthew J. (2019). Analyse de documents avec TF-IDF. *Programming historian*. [En ligne](https://programminghistorian.org/fr/lecons/analyse-de-documents-avec-tfidf)
- Bird, S. & Klein E. & Loper E. (1e édition 2009). *Natural language processing with pyton. Analyzing text with the natural language toolkit*. [En ligne](https://www.nltk.org/book/) (pour aller beaucoup plus loin) 
- [Cours de Python](https://github.com/PonteIneptique/cours-python) donné par Thibault Clérice à l'École des Chartes jusqu'en 2021-2022. Un cours trèèèèès complet qui va jusqu'au développement d'une appli Web.
